# AWS ParallelCluster Cost Estimation

When you create a cluster, all compute resources are tagged with the following tags
- ClusterName: name of the cluster
- QueueName: name of the queue

Those tags will help us identify overall cost of the cluster and the compute nodes of each queue in [AWS Cost and Usage Report (CUR)](https://docs.aws.amazon.com/cur/latest/userguide/what-is-cur.html). Because those tags are not default cost allocation tags, you will need to enable them from the billing console first. Usually it takes within 24 hours for those cost allocation tags to be activated.

However, when you operating a cluster which is shared among different users to run different jobs with different accounts, CUR will not be able to give you the level of breakdown to the job level. 

In this notebook, we will walk through how you can use the SLURM accounting to allocate costs (estimated) base on jobs, users and accounts (not AWS account , but "account" parameter you use when submmiting a slurm job). 

We will use data from the following sources for cost allocation:
- CUR data: using Amazon Athena to query the CUR datalake
- SLURM accounting data: using JDBC connections to the MySQL database, which backs the SLURMDBD data store

Assumptions:
- You have completed pcluster-athena++ notebook and has ran a few simulations or submitted some slurm jobs to the cluster at least one day before. 
- You have enabled the CUR and created the CUR datalake 
 


In [ ]:
import boto3
import botocore
import json
import time
import os
import base64
import pandas as pd
import importlib
import project_path # path to helper methods
from lib import workshop
from botocore.exceptions import ClientError
from IPython.display import display


session = boto3.session.Session()
region = session.region_name

my_account_id = boto3.client('sts').get_caller_identity().get('Account')

# The following 3 parameters are carried over from from pcluster-athena++ notebook. Please change them accordingly
# unique name of the pcluster. 
# This is the name of the cluster - default to "parallelcluster" - in this excercise, we default the cluster name to "parallelcluster"
# If you are using sacctmgr to track accounting for multiple clusters, this could be set differently

pcluster_name = 'myPC5c'
CLUSTERNAME="parallelcluster"


#CLUSTERNAME="mypc6g"
#pcluster_name = 'myPC6g'


# the rds for the Slurmdbd datastore. We will use a MySQL server as the data store. Server's hostname, username, password will be saved in a secret in Secrets Manager
rds_secret_name = 'slurm_dbd_credential'
db_name = 'pclusterdb'


# the CUR database and table names in the CUR datalake. Please see  AWS Cost and Usage Report (CUR) manual to find out how to set it up
# please chagne them to the name of your catalog database and table
cur_db = 'athenacurcfn_my_main_cur_in_athena'
cur_table = 'my_main_cur_in_athena'

# this bucket is used for storing Athena query results
bucket_prefix = pcluster_name.lower()+'-accounting-'+my_account_id

# use the bucket prefix as name, don't use uuid suffix
bucket_name = workshop.create_bucket(region, session, bucket_prefix, False)
print(bucket_name)

path_name = 'athena'

# we will look at the specific month
cur_year = '2021'
cur_month = '5'


In [ ]:
### assuem you have created a database secret in SecretManager with the name "slurm_dbd_credential"
def get_slurm_dbd_rds_secret():
    secret_name = rds_secret_name
    region_name = "us-east-1"

    # Create a Secrets Manager client
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return secret
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return decoded_binary_secret    
    
# the response is a json {"username": "xxxx", "password": "xxxx", "engine": "mysql", "host": "xxxx", "port": "xxxx", "dbInstanceIdentifier", "xxxx"}
rds_secret = json.loads(get_slurm_dbd_rds_secret())


In [ ]:
import boto3
import base64
import time
from botocore.exceptions import ClientError
from IPython.display import HTML, display

slurm_user = 'slurm'

def display_table(data):
    html = "<table>"
    for row in data:
        html += "<tr>"
        for field in row:
            html += "<td><h4>%s</h4><td>"%(field)
        html += "</tr>"
    html += "</table>"
    display(HTML(html))

###
# Retrieve the slurm_token from the SecretManager
#
def get_secret():
    secret_name = "slurm_token_{}".format(pcluster_name)
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
    except ClientError as e:
        print("Error", e)
        raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            secret = get_secret_value_response['SecretString']
            return secret
        else:
            decoded_binary_secret = base64.b64decode(get_secret_value_response['SecretBinary'])
            return decoded_binary_secret

###
# Retrieve the token and inject into the header for JWT auth
#
def update_header_token():
    token = get_secret()
    post_headers = {'X-SLURM-USER-NAME':slurm_user, 'X-SLURM-USER-TOKEN': token, 'Content-type': 'application/json', 'Accept': 'application/json'}
    get_headers = {'X-SLURM-USER-NAME':slurm_user, 'X-SLURM-USER-TOKEN': token, 'Content-type': 'application/x-www-form-urlencoded', 'Accept': 'application/json'}
    return [post_headers, get_headers]

###
# Convert response into json
#
def convert_response(resp):
    resp_str = resp.content.decode('utf-8')
    return json.loads(resp_str)



###
# Print a json array in table format
# input: headers [json attribute name, ... ]
# input: a - array of json objects
def print_table_from_json_array(headers, a):
    # add headers as the first row.
    t = [headers]
    for item in a:
        result = []
        for h in headers:
            result.append(item[h])
        t.append(result)
    display_table(t)

def print_table_from_dict(headers, d):
    result = list()
    for k,v in d.items():
        result.append(v)
    print_table_from_json_array(headers, result)
        

### 
# wrapper for get
#
def get_response_as_json(base_url):
    _, get_headers = update_header_token()
    resp = requests.get(base_url, headers=get_headers)
    if resp.status_code != 200:
        # This means something went wrong.
        print("Error" , resp.status_code)

    return convert_response(resp)

### 
# wrapper for post
#
def post_response_as_json(base_url, data):
    post_headers, _ = update_header_token()
    resp = requests.post(base_url, headers=post_headers, data=data)
    if resp.status_code != 200:
        # This means something went wrong.
        print("Error" , resp.status_code)

    return convert_response(resp)

###
# Epoch time conversion
#
def get_localtime(t):
    return time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(t))


## Accounting 
The Slurm account information is managed by slurmdbd process and stored in a data store( local file or a relational database). In our setup, we use an AWS RDS MySQL database, which has IAM authentication enabled and is running in the save VPC as the ParallelCluster. We can access the database from this notebook. 


In [ ]:
### get the root ca and install mysql.connector
# only need to do this once
#

#!wget https://s3.amazonaws.com/rds-downloads/rds-ca-2019-root.pem
#!pip install mysql.connector


In [ ]:
import mysql.connector
from mysql.connector.constants import ClientFlag
import sys
import boto3
import os


def get_sacct_as_table_for_cluster(rds_secret, cluster_name):
    ENDPOINT= rds_secret['host']
    PORT=rds_secret['port']
    USER=rds_secret['username']
    PASS=rds_secret['password']
    DBNAME="slurm_acct_db"


    ## use these of your want to use IAM authentication
    #os.environ['LIBMYSQL_ENABLE_CLEARTEXT_PLUGIN'] = '1
    #session = boto3.Session()
    #client = boto3.client('rds')
    #token = client.generate_db_auth_token(DBHostname=ENDPOINT, Port=PORT, DBUsername=USR, Region=REGION)

    config = {
        'user': USER,
        'password': PASS,  
        'host': ENDPOINT,
        'port': PORT,
        'database': DBNAME,
    # needed for IAM authentication
    #    'client_flags': [ClientFlag.SSL],
    #    'ssl_ca': 'rds-ca-2019-root.pem',
    }

    table_headers=['job_db_inx', 'account', 'cpus_req', 'job_name', 'id_job', 'nodelist', 'partition', 'time_submit', 'time_start', 'time_end', 'duration(s)', 'work_dir']
    table = []

    # partition is a reserved key word in mysql, need to use back tick ` around it
    try:
        conn =  mysql.connector.connect(**config)
        cur = conn.cursor()
        cur.execute("""SELECT job_db_inx, account, cpus_req, job_name, id_job, nodelist, `partition`, time_submit, time_start, time_end, work_dir from {}_job_table""".format(cluster_name))
        query_results = cur.fetchall()
    except Exception as e:
        print("Database connection failed due to {}".format(e)) 
        raise

    #job_table_header =[(0,'job_id_inx'), (3, 'account'), ()) 
    for r in query_results:
        l = list(r)
        # add a duration before the last element
        l.append(l[10])
        #duration
        l[10] = -1  if ( l[9]==0) else (l[9]-l[8])
        l[7] = get_localtime(l[7])
        l[8] = get_localtime(l[8])
        l[9] = get_localtime(l[9])
        table.append(l)
    
    return table, table_headers

    
def display_allocated_cost(table, table_headers, daily_cluster_df_cost) :
    df = pd.DataFrame(table, columns=table_headers)
    # convert time_start from string to datetime

    df['time_start'] = pd.to_datetime(df['time_start'])

    # drop id_job and job_db_inx - sum of those not useful
    df = df.drop(columns=['job_db_inx', 'id_job'])

    # sum calculation durations fot account, partition, job_name per day
    # partition and time_start is now the index ,need to reset_index to keep the partition, time_start as columns 
    agg_df= df.groupby(['account', 'partition', 'job_name', df['time_start'].dt.date]).sum().reset_index()

    # partition and time_start is now the index 
    agg_df_daily= agg_df.groupby(['partition', agg_df['time_start']]).sum()

    allocations = []
    costs = []
    costs_total = []
    for idx, row in agg_df.iterrows():
        loc_idx_queue_datetime = (row['partition'], row['time_start'])
        loc_idx_datetime = (row['time_start'])
        arow = row['duration(s)']/agg_df_daily.loc[[loc_idx_queue_datetime], 'duration(s)']
        try:
            row_cost = arow[0]*daily_cluster_df_cost.loc[[loc_idx_datetime], 'compute_cost']
            row_total_cost = arow[0]*daily_cluster_df_cost.loc[[loc_idx_datetime], 'cost']
        except:
            #CUR did not have the queue information in some of the dates. 
            row_cost = []
            row_total_cost = []
            row_cost.append(0)
            row_total_cost.append(0)

        allocations.append(arow[0])
        costs.append(row_cost[0])
        costs_total.append(row_total_cost[0])

    agg_df['allocations'] = allocations
    agg_df['compute_cost'] = costs
    agg_df['total_cost'] = costs_total
    agg_indexed_df =agg_df.set_index(['time_start', 'partition']).sort_values(['time_start', 'partition'])
    display(agg_indexed_df)
    
    return agg_df

### Calculate the cost allocation for each job

Slurm accounting will provide duration in seconds for each account and job_name.  

First, call the PClusterCostEstimator to get the daily spending of each queue from the Cost And Usage Report (CUR), using PClusterCostEstimator helper class. This daily cost only include the cost of compute nodes (not the head node)

In [ ]:
import sys
# this is used during developemnt, to reload the module after a change in the module
try:
    del sys.modules['pcluster_cost_estimator']
except:
    #ignore if the module is not loaded
    print('Module not loaded, ignore')
    
from pcluster_cost_estimator import PClusterCostEstimator



pce = PClusterCostEstimator(cur_db, cur_table, bucket_name, path_name)
daily_queue_df = pce.cluster_daily_per_queue_month(pcluster_name, cur_year, cur_month).reset_index()

daily_queue_df['time_start'] = pd.to_datetime(daily_queue_df['time_start'])
daily_queue_df = daily_queue_df.set_index(['partition','time_start' ])
                         
#update the index to queue_name+datetime
display(daily_queue_df)



In [ ]:
daily_compute_df = daily_queue_df.groupby(['time_start']).sum()

### Compare daily cluster cost with compute cost

CUR will provide you with total cost of : 
- Cluster - with ClusterName = 'my_cluster_name'
- Compute cost - with ClusterName = 'my_cluster_name' and QueueName = [queue_names]

Compute cost doesn't incldue the cost of the head node.

In [ ]:

for cn in ['parallelcluster', 'mypc6g']:
    print(f"For cluster {cn}:")
    table, table_headers = get_sacct_as_table_for_cluster(rds_secret, cn)
    agg_df = display_allocated_cost(table, table_headers, daily_cluster_df_cost)



In [ ]:

# show the total allocated cost by date (add all queue spends together)
daily_queue_df_cost = agg_df.groupby(level=0).sum()
#display(daily_queue_df_cost)

daily_cluster_df_cost = pce.cluster_daily_per_month(pcluster_name, cur_year, cur_month)
daily_cluster_df_cost['compute_cost'] = daily_compute_df['cost']
display(daily_cluster_df_cost)

## Clean up

In [ ]:
workshop.delete_bucket_completely(my_bucket_name)